## Guidelines

This intended use of this notebook is as an *independent* source for results used in AEIC test cases. As such, no code may be imported from AEIC modules here. Ideally, only Python standard library imports should be used.

For each test case, there should be an **Independent implementation** section (with an implementation of whatever is being tested that is as independent as possible from the code in AEIC), a **Test data** section and a **Test case evaluation** section. The idea is that the contents of the **Test data** and **Test case evaluation** sections can be copied and pasted into a real Python test function but that the implementation getting from one to the other is independent of what happens in the AEIC code.

(The word "independent" here is doing a lot of work. In an ideal world, the independent implementation would be written by someone other than the person writing the implementation in AEIC, without too much communication other than "I'm implementing equation (X) from this paper", but that's obviously impractical in many cases. If you find yourself needing to implement both the test case generation and the actual AEIC code, try to split your mind and keep them independent!)

In [61]:
import math

### 1. Sea-level-static fuel flow

The `get_SLS_equivalent_fuel_flow` function converts fuel flow at altitude to its equivalent at sea level using calculations from the Boeing Fuel Flow Method 2 (BFFM2) from DuBois, D. & Paynter, G. (2006). *Fuel Flow Method 2 for Estimating Aircraft Emissions*. SAE Technical Paper 2006-01-1987.

Here we generate independent test data using an implementation of the method taken directly from equation (40) in the paper.

#### Independent implementation

Define some constants:

In [62]:
# Sea level pressure [Pa].
pressure_SL = 101325.0

# Sea level temperature [K].
temperature_SL = 288.15

# Ratio of specific heats at constant pressure and constant volume for dry air.
kappa = 1.4

# Specific gas constant for dry air [J kg-1 K-1].
R_air = 287.05287

Define a function to calculate equation (40) from the paper (we need fuel flow, pressure, temperature and airspeed, since the equation makes use of the Mach number):

In [63]:
def fuel_flow_to_sls(fuel_flow: float, pressure: float, temperature: float, air_speed: float) -> float:
    # Mach number.
    M = air_speed / math.sqrt(kappa * R_air * temperature)

    # Non-dimensional pressure and temperature.
    delta_amb = pressure / pressure_SL
    theta_amb = temperature / temperature_SL

    # Equation (40) from paper.
    # print(f'M = {M}  delta_amb = {delta_amb}  theta_amb = {theta_amb}')
    return fuel_flow * theta_amb**3.8 / delta_amb * math.exp(0.2 * M**2)

#### Test data

Some of this test data is generated from the `AtmosphericState` class in the AEIC code.

In [64]:
# Reference altitudes [m] along test trajectory.
altitudes = [0.0, 1500.0, 6000.0, 11000.0, 9000.0, 2000.0]

# Temperatures [K] and pressures [Pa] along test trajectory (calculated from ISA atmosphere).
temperatures = [288.15, 278.4, 249.15, 216.65, 229.65, 275.15]
pressures = [101325.0, 84555.9940737564, 47181.0021852292, 22632.0400950078, 30742.4326120969, 79495.201934051]

# Total fuel flows along test trajectory [kg/s].
full_fuel_flows = [0.3, 0.35, 0.55, 0.65, 0.5, 0.32]

# Per-engine fuel flows along test trajectory [kg/s].
number_of_engines = 2
fuel_flows = [ff / number_of_engines for ff in full_fuel_flows]

# Airspeed along test trajectory [m/s].
airspeeds = [120.0, 150.0, 190.0, 210.0, 180.0, 140.0]

#### Evaluate test case

In [65]:
results = list(map(fuel_flow_to_sls, fuel_flows, pressures, temperatures, airspeeds))

In [66]:
results

[0.15377734749955685,
 0.19154479109277428,
 0.3652574468094632,
 0.5447580164850215,
 0.37317567328909007,
 0.17729854838504117]

Rounded results for use in test:

In [67]:
[round(r, 6) for r in results]

[0.153777, 0.191545, 0.365257, 0.544758, 0.373176, 0.177299]

# 2. BFFM2 Implementation same as that in MATLAB AEIC

The Boeing Fuel flow 2 method (from the Boeing Fuel Flow Method 2 (BFFM2) from DuBois, D. & Paynter, G. (2006). *Fuel Flow Method 2 for Estimating Aircraft Emissions*. SAE Technical Paper 2006-01-1987) calculates the emissions index of NOx from fuel flow, ambient temperatures and pressures, as well as reference fuel flow and EI NOx points that it interpolates from.

The below implementation recreates while also using the same matlab code in AEICv2 (https://github.mit.edu/LAE/PW_EEI/blob/main/AEIC/Model_Files/noxEIsFunc.m) or (/home/aditeya/SM_Thesis/PW_EEI/AEIC/Model_Files/noxEIsFunc.m)

In [68]:
import numpy as np

def BFFM_FROM_MATLAB(sls_equiv_fuel_flow: np.ndarray, #called fuelfactor in matlab code
    EI_NOx_matrix: np.ndarray,
    fuelflow_performance: np.ndarray, 
    Tamb: np.ndarray,
    Pamb: np.ndarray):
        
    # Make fuel flow small if <= 0
    fuelflow_performance[fuelflow_performance <= 0] = 0.01

    # MATLAB: zeros(length(fuelflow_KGperS), 2)
    NOx_linfits = np.zeros((max(fuelflow_performance.shape), 2))

    # MATLAB size(...,1) stuff
    ff = fuelflow_performance if fuelflow_performance.ndim > 1 else fuelflow_performance[None, :]
    eim = EI_NOx_matrix if EI_NOx_matrix.ndim > 1 else EI_NOx_matrix[None, :]
    ffactor = sls_equiv_fuel_flow if sls_equiv_fuel_flow.ndim > 1 else sls_equiv_fuel_flow[None, :]
    nrows = ff.shape[0]
    # Interpolation:
    for i in range(nrows):
        NOx_linfits[i, :] = np.polyfit(
            np.log10(ff[i, :]),
            np.log10(eim[i, :]),
            1
        )
    
    NOxEI = np.zeros_like(ffactor)

    for i in range(nrows):
        NOxEI[i, :] = 10 ** (np.log10(ffactor[i, :]) * NOx_linfits[i, 0] + NOx_linfits[i, 1])
        theta_amb = Tamb / 288.15
        delta_amb = Pamb / 101325.0
        Pamb_psia = delta_amb * 14.696
        # Humidity correction
        beta = (
            7.90298 * (1.0 - (373.16) / (Tamb + 0.01)) + 3.00571 +
            5.02808 * np.log10((373.16) / (Tamb + 0.01)) +
            1.3816e-7 * (1.0 - (10.0 ** (11.344 * (1.0 - ((Tamb + 0.01) / 373.16))))) +
            8.1328e-3 * ((10.0 ** (3.49149 * (1.0 - (373.16) / (Tamb + 0.01)))) - 1.0)
        )

        Pv = 0.014504 * (10.0 ** beta)
        phi = 0.6
        omega = (0.62198 * phi * Pv) / (Pamb_psia - (phi * Pv))
        H = -19.0 * (omega - 0.0063)

        NOxEI[i, :] = NOxEI[i, :] * np.exp(H) * (((delta_amb ** 1.02) / (theta_amb ** 3.3)) ** 0.5)

    # NOx breakdown
    lowLimit = (fuelflow_performance[0] + fuelflow_performance[1]) / 2
    approachLimit = (fuelflow_performance[1] + fuelflow_performance[2]) / 2

    thrustCat = (sls_equiv_fuel_flow <= lowLimit) * 2 + (sls_equiv_fuel_flow > approachLimit) * 1
    thrustCat = thrustCat + (thrustCat == 0) * 3

    # Define proportion bounds for different thrust categories
    # HONO (HONO/NOy)
    honoHnom = 0.75
    honoHupp = 1.0
    honoHlow = 0.5

    honoLnom = 4.5
    honoLupp = 7.0
    honoLlow = 2.0

    honoAnom = 4.5
    honoAupp = 7.0
    honoAlow = 2.0

    # NO2 (NO2/NOy) assuming known values are NO2/(NO+NO2) = NO2(NOy-HONO).
    # This is to ensure always sum to 1.
    no2Hnom = 7.5 * (100 - honoHnom) / 100
    no2Hupp = 10.0 * (100 - honoHupp) / 100
    no2Hlow = 5.0 * (100 - honoHlow) / 100

    no2Lnom = 86.5 * (100 - honoLnom) / 100
    no2Lupp = 98.0 * (100 - honoLnom) / 100
    no2Llow = 75.0 * (100 - honoLnom) / 100

    no2Anom = 16.0 * (100 - honoAnom) / 100
    no2Aupp = 20.0 * (100 - honoAnom) / 100
    no2Alow = 12.0 * (100 - honoAnom) / 100

    # NO (NO/NOy) [Nominal values]
    noHnom = 100 - honoHnom - no2Hnom
    noLnom = 100 - honoLnom - no2Lnom
    noAnom = 100 - honoAnom - no2Anom

    honoH = honoHnom
    honoL = honoLnom
    honoA = honoAnom

    no2H = no2Hnom
    no2L = no2Lnom
    no2A = no2Anom

    noH = 100 - honoH - no2H
    noL = 100 - honoL - no2L
    noA = 100 - honoA - no2A

    # Thrust specific breakdown
    honoProp = ((honoH * (thrustCat == 1)) + (honoL * (thrustCat == 2)) + (honoA * (thrustCat == 3))) / 100.0
    no2Prop  = ((no2H  * (thrustCat == 1)) + (no2L  * (thrustCat == 2)) + (no2A  * (thrustCat == 3))) / 100.0
    noProp   = ((noH   * (thrustCat == 1)) + (noL   * (thrustCat == 2)) + (noA   * (thrustCat == 3))) / 100.0

    NOEI = NOxEI * noProp
    NO2EI = NOxEI * no2Prop
    HONOEI = NOxEI * honoProp

    return NOxEI, NOEI, NO2EI, HONOEI, noProp, no2Prop, honoProp

In [69]:
# FROM EDB excel
EDB_fuel_flows = np.array([0.4, 0.8, 1.2, 1.8])
EDB_EI_NOx = np.array([30.0, 25.0, 20, 18.0])

# Temperatures [K] and pressures [Pa] along test trajectory (calculated from ISA atmosphere).
temperatures = np.array([288.15, 278.4, 249.15, 216.65, 229.65, 275.15])
pressures = np.array([101325.0, 84555.9940737564, 47181.0021852292, 22632.0400950078, 30742.4326120969, 79495.201934051])

# Total fuel flows along test trajectory [kg/s].
full_fuel_flows = np.array([0.3, 0.35, 0.55, 0.65, 0.5, 0.32])

# Per-engine fuel flows along test trajectory [kg/s].
number_of_engines = 2
fuel_flows = [ff / number_of_engines for ff in full_fuel_flows]

# Getting sls fuel flow from above function - is this allowed?
sls_equiv_fuel_flow = np.array(list((map(fuel_flow_to_sls, fuel_flows, pressures, temperatures, airspeeds))))


In [70]:
sls_equiv_fuel_flow

array([0.15377735, 0.19154479, 0.36525745, 0.54475802, 0.37317567,
       0.17729855])

In [71]:
results_BFFM = BFFM_FROM_MATLAB(sls_equiv_fuel_flow, EDB_EI_NOx, EDB_fuel_flows, temperatures, pressures)

In [72]:
results_BFFM

(array([[42.65302497, 39.87840171, 30.13039678, 22.9420127 , 27.77833904,
         40.95955377]]),
 array([[5.49904124, 5.14132294, 3.88456141, 2.95779899, 3.58132236,
         5.28071047]]),
 array([[35.2345976 , 32.94255069, 24.88996752, 18.95182314, 22.94699142,
         33.83566338]]),
 array([[1.91938612, 1.79452808, 1.35586786, 1.03239057, 1.25002526,
         1.84317992]]),
 array([0.128925, 0.128925, 0.128925, 0.128925, 0.128925, 0.128925]),
 array([0.826075, 0.826075, 0.826075, 0.826075, 0.826075, 0.826075]),
 array([0.045, 0.045, 0.045, 0.045, 0.045, 0.045]))

# 3. Test NOx speciation

The below implementation recreates while also using the same matlab code in AEICv2 (https://github.mit.edu/LAE/PW_EEI/blob/main/AEIC/Model_Files/noxEIsFunc.m) or (/home/aditeya/SM_Thesis/PW_EEI/AEIC/Model_Files/noxEIsFunc.m)

In [4]:
import numpy as np
def NOx_speciation_from_matlab():
    # -------------------------
    # NOx breakdown
    # -------------------------

    # HONO (HONO/NOy)
    honoHnom = 0.75
    honoHupp = 1
    honoHlow = 0.5

    honoLnom = 4.5
    honoLupp = 7
    honoLlow = 2

    honoAnom = 4.5
    honoAupp = 7
    honoAlow = 2

    # NO2 (NO2/NOy) assuming known values are NO2/(NO+NO2) = NO2(NOy-HONO).
    # This is to ensure always sum to 1.
    no2Hnom = 7.5 * (100 - honoHnom) / 100
    no2Hupp = 10 * (100 - honoHupp) / 100
    no2Hlow = 5 * (100 - honoHlow) / 100

    no2Lnom = 86.5 * (100 - honoLnom) / 100
    no2Lupp = 98 * (100 - honoLnom) / 100
    no2Llow = 75 * (100 - honoLnom) / 100

    no2Anom = 16 * (100 - honoAnom) / 100
    no2Aupp = 20 * (100 - honoAnom) / 100
    no2Alow = 12 * (100 - honoAnom) / 100

    # NO (NO/NOy) [Nominal values]
    noHnom = 100 - honoHnom - no2Hnom
    noLnom = 100 - honoLnom - no2Lnom
    noAnom = 100 - honoAnom - no2Anom

    honoH = honoHnom
    honoL = honoLnom
    honoA = honoAnom

    no2H = no2Hnom
    no2L = no2Lnom
    no2A = no2Anom

    noH = 100 - honoH - no2H
    noL = 100 - honoL - no2L
    noA = 100 - honoA - no2A

    # -------------------------
    # Thrust specific breakdown
    # -------------------------

    LTO_thrustCat = np.array([2,3,1,1])

    honoProp = ((honoH * (LTO_thrustCat == 1)) +
                (honoL * (LTO_thrustCat == 2)) +
                (honoA * (LTO_thrustCat == 3))) / 100

    no2Prop = ((no2H * (LTO_thrustCat == 1)) +
            (no2L * (LTO_thrustCat == 2)) +
            (no2A * (LTO_thrustCat == 3))) / 100

    noProp = ((noH * (LTO_thrustCat == 1)) +
            (noL * (LTO_thrustCat == 2)) +
            (noA * (LTO_thrustCat == 3))) / 100

    return honoProp, no2Prop, noProp


In [5]:
print(NOx_speciation_from_matlab())

(array([0.045 , 0.045 , 0.0075, 0.0075]), array([0.826075 , 0.1528   , 0.0744375, 0.0744375]), array([0.128925 , 0.8022   , 0.9180625, 0.9180625]))
